In [ ]:
import pandas as pd

# pass in column names for each CSV and read them using pandas. 
# Column names available in the readme file

#Reading users file:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']

users=pd.read_csv('/home/ranjani/Downloads/ml-100k/u.user', sep='|', names=u_cols, encoding='latin-1')

r_cols = ['user_id', 'movie id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('/home/ranjani/Downloads/ml-100k/u.data', sep='\t', names=r_cols, encoding='latin-1')

#Reading items file:
i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv('/home/ranjani/Downloads/ml-100k/u.item', sep='|', names=i_cols,
 encoding='latin-1')



In [32]:
print (users.shape)
users.head()


(943, 5)


,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [33]:

dataset=pd.merge(items, ratings)
print(dataset.head())

dataset1 = pd.merge(pd.merge(items, ratings),users)

   movie id       movie title release date  video release date  \
0         1  Toy Story (1995)  01-Jan-1995                 NaN   
1         1  Toy Story (1995)  01-Jan-1995                 NaN   
2         1  Toy Story (1995)  01-Jan-1995                 NaN   
3         1  Toy Story (1995)  01-Jan-1995                 NaN   
4         1  Toy Story (1995)  01-Jan-1995                 NaN   

                                            IMDb URL  unknown  Action  \
0  http://us.imdb.com/M/title-exact?Toy%20Story%2...        0       0   
1  http://us.imdb.com/M/title-exact?Toy%20Story%2...        0       0   
2  http://us.imdb.com/M/title-exact?Toy%20Story%2...        0       0   
3  http://us.imdb.com/M/title-exact?Toy%20Story%2...        0       0   
4  http://us.imdb.com/M/title-exact?Toy%20Story%2...        0       0   

   Adventure  Animation  Children's       ...        Musical  Mystery  \
0          0          1           1       ...              0        0   
1          0      

In [34]:
ratings_total = dataset.groupby('movie title').size()
print(ratings_total.head())

movie title
'Til There Was You (1997)      9
1-900 (1994)                   5
101 Dalmatians (1996)        109
12 Angry Men (1957)          125
187 (1997)                    41
dtype: int64


In [35]:
ratings_mean = (dataset.groupby('movie title'))['movie title','rating'].mean()
print(ratings_mean.head())

                             rating
movie title                        
'Til There Was You (1997)  2.333333
1-900 (1994)               2.600000
101 Dalmatians (1996)      2.908257
12 Angry Men (1957)        4.344000
187 (1997)                 3.024390


In [36]:
#modify the dataframes so that we can merge the two
ratings_total = pd.DataFrame({'movie title':ratings_total.index,
'total ratings': ratings_total.values})
ratings_mean['movie title'] = ratings_mean.index

In [37]:
final = pd.merge(ratings_mean, ratings_total).sort_values(by = 'total ratings',
ascending= False)
print(final.head())


        rating                movie title  total ratings
1398  4.358491           Star Wars (1977)            583
333   3.803536             Contact (1997)            509
498   4.155512               Fargo (1996)            508
1234  4.007890  Return of the Jedi (1983)            507
860   3.156701           Liar Liar (1997)            485


/home/ranjani/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3267: FutureWarning: 'movie title' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  exec(code_obj, self.user_global_ns, self.user_ns)


In [38]:
#Top ten movies among all users
final = final[:200].sort_values(by = 'rating',
ascending = False)
print(final.head(10))


        rating                             movie title  total ratings
1281  4.466443                 Schindler's List (1993)            298
273   4.456790                       Casablanca (1942)            243
1317  4.445230        Shawshank Redemption, The (1994)            283
1215  4.387560                      Rear Window (1954)            209
1572  4.385768              Usual Suspects, The (1995)            267
1398  4.358491                        Star Wars (1977)            583
303   4.292929                     Citizen Kane (1941)            198
1507  4.292237            To Kill a Mockingbird (1962)            219
1102  4.291667  One Flew Over the Cuckoo's Nest (1975)            264
1329  4.289744        Silence of the Lambs, The (1991)            390


In [ ]:
#Pref contains the list of movies seen by each user

pref={}
Users=users['user_id'].values.tolist()
for user in Users:
    pref[user]=ratings[ratings['user_id']==user]

In [170]:
#computes the similarity etween two users based on number of common movies.
def sim_jaccard(p1, p2):
    numerator = 0
    difference = 0
    for movie in pref[p1]['movie id'].values:
        if movie in pref[p2]['movie id'].values:
            numerator = numerator + 1
        else:
            difference = difference + 1
    denominator = len(pref[p1].keys()) + difference        
    return numerator/denominator 


In [ ]:
#Given two users and a movie, it checks how similar user 2 is to user 1 and returns a suitable rating based on
#weighted mean of the ratings.
def sim_ratings(movie,guser):
    print ("hi")
    i=1
    rating=0
    users1=ratings[ratings['movie id']==movie]
    for user in users1['user_id'].values:
        sim=sim_jaccard(guser,user)
        s=pref[user].loc[pref[user]['movie id']== movie ,'rating'].values
        simrat=sim*s[0]
        rating+=simrat
        i+=1
    rating=rating/i
    return rating
        
    

In [219]:
#taking user name as input, checks which users have rated each movie and computes a rating for each movie based on
#the two users' similarity and the second users' rating.

user_inp=int(input('Enter user ID:'))

movies=items.loc[~items['movie id'].isin(pref[int(user_inp)]['movie id'].values)]

movies['simratings']= movies.apply (lambda row: sim_ratings(row['movie id'],user_inp), axis=1)

Enter user ID:3


/home/ranjani/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [ ]:
#Sorting the movies based on similarity ratings and displaying top 10.
top10=movies.sort_values( ["simratings"], ascending = False )[:10]

print(top10[['movie title','simratings']])

                                      movie title  simratings
813                 Great Day in Harlem, A (1994)   10.681818
856                      Paris Was a Woman (1995)    6.409091
850   Two or Three Things I Know About Her (1966)    4.621333
866                  Whole Wide World, The (1996)    4.619242
1612                            Tokyo Fist (1995)    4.105263
1242                           Night Flier (1997)    4.051153
1667                    Wedding Bell Blues (1996)    3.937500
1669                               Tainted (1998)    3.937500
1666                        Next Step, The (1995)    3.937500
916                         Mercury Rising (1998)    3.894754
